# Librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from mpl_toolkits.mplot3d import Axes3D
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score

# Carga de los datos 

Los datos vienen con su respectiva etiqueta correspondiente a cada cluster.


In [5]:
positivos = pd.read_csv('..\Datos\positivos_cluster.csv')
negativos = pd.read_csv('..\Datos\\negativos_cluster.csv')

# Latent Dirichlet Allocation (LDA)

Este modelo nos servira para interpretar que tipo de comentarios agrupa cada cluster. De esta forma, podremos nombrar cada cluster con un nombre descriptivo.

In [6]:
import nltk
nltk.download('stopwords')
stopword_es = nltk.corpus.stopwords.words('spanish')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import gensim

In [10]:
from gensim import models, corpora
from gensim.utils import simple_preprocess

stopwords = set(['y','que','https','el','lo','un','en','la','del'
'tu','las','ya','por','esa','las','eso','qué','sin','su','están','esto'
'es','una','más','mas','tiene','q','t','co','de','para','esto','los','son','con'
'se','le','al','cuando','e','user','como','es','si','tienen','se','con','del'
'pero','sus','te','muy','del','me','mucho','no','he','pero','da','nos','ademas','hace','todo'
])

stopwords.update(stopword_es)

tokens = [
    [word for word in simple_preprocess(comentario, deacc=True) if word not in stopwords]
    for comentario in positivos['comentario']
]

dictionary = corpora.Dictionary(tokens)
#no_below -> tokens en al menos X documentos los que no se seran eliminados y que no esten contenidos en mas del 50% de los documentos (no_above)
dictionary.filter_extremes(no_below=50, no_above=0.8)

# Crear el corpus en el formato que LDA espera
corpus = [dictionary.doc2bow(token) for token in tokens]


# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

ModuleNotFoundError: No module named 'gensim'

In [29]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [105]:
dic = [[(id2word[i], freq) for i, freq in doc] for doc in corpus[:5]]


In [34]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 6
chunksize = 2000 # numero de documentos procesados a la vez
passes = 20 # cantidad de veces que se pasa por el total del corpus
iterations = 400 # cantidad de iteraciones durante el entrenamiento
eval_every = None  # Don't evaluate model perplexity, takes too much time.

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state=1,
    minimum_probability=0.3
)

In [102]:
dictionary.id2token


{0: 'auxiliares',
 1: 'bien',
 2: 'explico',
 3: 'inquietudes',
 4: 'preocupo',
 5: 'profesor',
 6: 'clases',
 7: 'materia',
 8: 'agradecido',
 9: 'ayudarnos',
 10: 'contenidos',
 11: 'curso',
 12: 'docente',
 13: 'equipo',
 14: 'interesante',
 15: 'largo',
 16: 'realidad',
 17: 'relacion',
 18: 'semestre',
 19: 'auxs',
 20: 'excelentes',
 21: 'aunque',
 22: 'bastante',
 23: 'capacidad',
 24: 'catedras',
 25: 'clase',
 26: 'comun',
 27: 'ejemplos',
 28: 'ejercicios',
 29: 'etc',
 30: 'explica',
 31: 'explicaciones',
 32: 'explican',
 33: 'explicar',
 34: 'frente',
 35: 'grandes',
 36: 'gusto',
 37: 'hizo',
 38: 'ideas',
 39: 'importantes',
 40: 'ingenieria',
 41: 'mediante',
 42: 'mejor',
 43: 'metodos',
 44: 'muchas',
 45: 'personalmente',
 46: 'plan',
 47: 'primer',
 48: 'problemas',
 49: 'programas',
 50: 'ramo',
 51: 'repaso',
 52: 'resolver',
 53: 'tiempo',
 54: 'unico',
 55: 'veces',
 56: 'vez',
 57: 'videos',
 58: 'agradece',
 59: 'completo',
 60: 'disponibilidad',
 61: 'gran',


In [106]:
vis = gensimvis.prepare(model, corpus, dic)
pyLDAvis.display(vis)


AttributeError: 'list' object has no attribute 'token2id'

In [35]:
from pprint import pprint


pprint(model.print_topics())
doc_lda = model[corpus]

[(0,
  '0.086*"siempre" + 0.084*"buena" + 0.072*"auxiliares" + 0.060*"dudas" + '
  '0.057*"disposicion" + 0.031*"clases" + 0.021*"profesora" + '
  '0.021*"responder" + 0.021*"buenos" + 0.019*"resolver"'),
 (1,
  '0.187*"profe" + 0.186*"bien" + 0.076*"explica" + 0.058*"aux" + '
  '0.057*"super" + 0.028*"explicaba" + 0.026*"ensena" + 0.022*"gusta" + '
  '0.020*"simpatico" + 0.013*"auxs"'),
 (2,
  '0.105*"profesor" + 0.072*"buen" + 0.047*"estudiantes" + 0.046*"aprendizaje" '
  '+ 0.040*"alumnos" + 0.031*"nota" + 0.030*"ensenar" + 0.027*"excelente" + '
  '0.023*"docente" + 0.022*"ambiente"'),
 (3,
  '0.071*"curso" + 0.055*"docente" + 0.050*"ramo" + 0.042*"equipo" + '
  '0.041*"gracias" + 0.039*"semestre" + 0.035*"mejor" + 0.023*"muchas" + '
  '0.014*"mejores" + 0.014*"hizo"'),
 (4,
  '0.061*"clases" + 0.046*"materia" + 0.025*"clase" + 0.023*"bastante" + '
  '0.022*"ejercicios" + 0.021*"contenidos" + 0.020*"forma" + 0.019*"manera" + '
  '0.015*"entender" + 0.014*"profesor"'),
 (5,
  '0.044*

# LDA Mallet

In [25]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [28]:
from gensim.models.wrappers import LdaMallet

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [23]:
mallet_path = '..\Mallet-202108-bin.tar.gz' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('nCoherence Score: ', coherence_ldamallet)

AttributeError: module 'gensim.models' has no attribute 'wrappers'

# Evaluación

In [36]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [40]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=tokens, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


nCoherence Score:  0.5534109666355468


# Visualización

In [95]:
dictionary.id2token

{0: 'auxiliares',
 1: 'bien',
 2: 'explico',
 3: 'inquietudes',
 4: 'preocupo',
 5: 'profesor',
 6: 'clases',
 7: 'materia',
 8: 'agradecido',
 9: 'ayudarnos',
 10: 'contenidos',
 11: 'curso',
 12: 'docente',
 13: 'equipo',
 14: 'interesante',
 15: 'largo',
 16: 'realidad',
 17: 'relacion',
 18: 'semestre',
 19: 'auxs',
 20: 'excelentes',
 21: 'aunque',
 22: 'bastante',
 23: 'capacidad',
 24: 'catedras',
 25: 'clase',
 26: 'comun',
 27: 'ejemplos',
 28: 'ejercicios',
 29: 'etc',
 30: 'explica',
 31: 'explicaciones',
 32: 'explican',
 33: 'explicar',
 34: 'frente',
 35: 'grandes',
 36: 'gusto',
 37: 'hizo',
 38: 'ideas',
 39: 'importantes',
 40: 'ingenieria',
 41: 'mediante',
 42: 'mejor',
 43: 'metodos',
 44: 'muchas',
 45: 'personalmente',
 46: 'plan',
 47: 'primer',
 48: 'problemas',
 49: 'programas',
 50: 'ramo',
 51: 'repaso',
 52: 'resolver',
 53: 'tiempo',
 54: 'unico',
 55: 'veces',
 56: 'vez',
 57: 'videos',
 58: 'agradece',
 59: 'completo',
 60: 'disponibilidad',
 61: 'gran',


In [1]:
## Visualizacion de los topicos
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(model, corpus, dictionary.id2token)
p

ModuleNotFoundError: No module named 'pyLDAvis'